## Exercise 15: Implement an Oracle for Simon’s Algorithm
1. Implement an oracle for Simon’s Algorithm with a hidden periodicity s. For example, the function f(x) = f(x ⊕ s) for some secret string s = 101.
2. Create a quantum circuit for Simon’s Algorithm using the oracle and measure the result to find the hidden string s.

Expected Outcome: The measurement results will reveal the hidden string s after solving the system of equations generated from the quantum measurements.

[Hints:] The oracle can be implemented as a controlled-X gate (CX) with an input qubit
controlling a corresponding output qubit. After querying the oracle, apply Hadamard gates
and then measure the result.

In [1]:
from qiskit import *
from qiskit.primitives import StatevectorSampler  
from qiskit.visualization import plot_histogram
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector
from numpy import sqrt
import numpy as np
from qiskit.primitives import Sampler
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [2]:
def simon_function(s: str):
    
    n = len(s)
    qc = QuantumCircuit(2 * n)

    # Define a random permutation of all n bit strings. This permutation will effectively hide the string s.
    pi = np.random.permutation(2**n)
    query_gate = np.zeros((4**n, 4**n))
    for x in range(2**n):
        for y in range(2**n):
            z = y ^ pi[min(x, x ^ int(s, 2))]
            query_gate[x + 2**n * z, x + 2**n * y] = 1

    # Our circuit has just this one query gate
    qc.unitary(query_gate, range(2 * n))
    return qc

In [3]:
from qiskit_aer import AerSimulator
from qiskit import ClassicalRegister

def simon_measurements(problem: QuantumCircuit, k: int):
    
    n = problem.num_qubits // 2

    qc = QuantumCircuit(2 * n, n)
    qc.h(range(n))
    qc.compose(problem, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))

    result = AerSimulator().run(qc, shots=k, memory=True).result()
    return result.get_memory()

In [4]:
import numpy as np
import galois

def simon_algorithm(problem: QuantumCircuit):
    
    measurements = simon_measurements(problem, k=problem.num_qubits // 2 + 10)
    print("Measurement results:")
    display(measurements)

    matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)

    null_space = galois.GF(2)(matrix).null_space()
    print("Null space:")
    display(null_space)

    print("Guess for hidden string s:")
    if len(null_space) == 0:
        return "0" * len(measurements[0])
    return "".join(np.array(null_space[0]).astype(str))

In [5]:
display(simon_algorithm(simon_function("101")))

Measurement results:


['111',
 '101',
 '101',
 '101',
 '101',
 '010',
 '111',
 '000',
 '010',
 '111',
 '101',
 '101',
 '111']

Null space:


GF([[1, 0, 1]], order=2)

Guess for hidden string s:


'101'